# **XG Boost tuned model**

For this competition, the objective is to use the test dataset to predict whether a rookie NBA player will be playing in the NBA after 5 years. In a never ending quest to find the next Michael Jordan or Kobe Bryant, NBA teams are looking to invest in young players and want to know, with evidence-based analysis, whether their investments will pay off. This vignette will detail the steps undertaken to create a predictive model and the accuracy of the model against the stated aims.

In [12]:
#Initial imports
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import os
import sys
sys.path.append(os.path.abspath('..'))
from src.common_lib import DataReader, NBARawData
import xgboost

## **Data input and cleansing**

The nominated dataset for training and testing was supplied by the kaggle competition. The training dataset is called train.csv and the test dataset is called test.csv. 

In [181]:
#Load dataset using common function DataReader.read_data()
data_reader = DataReader()
# Load Raw Train Data
df_train = data_reader.read_data(NBARawData.TRAIN)
# Load Test Raw Data
df_test = data_reader.read_data(NBARawData.TEST)

In [182]:
#For train dataframe, remove redundant column 'Id_old'
cols_drop = ["Id", "Id_old"]
df_train.drop(cols_drop, axis=1, inplace=True)
df_train.columns = df_train.columns.str.strip()
df_train.describe

<bound method NDFrame.describe of       GP   MIN   PTS  FGM  FGA   FG%  3P Made  3PA   3P%  FTM  FTA   FT%  \
0     80  24.3   7.8  3.0  6.4  45.7      0.1  0.3  22.6  2.0  2.9  72.1   
1     75  21.8  10.5  4.2  7.9  55.1     -0.3 -1.0  34.9  2.4  3.6  67.8   
2     85  19.1   4.5  1.9  4.5  42.8      0.4  1.2  34.3  0.4  0.6  75.7   
3     63  19.1   8.2  3.5  6.7  52.5      0.3  0.8  23.7  0.9  1.5  66.9   
4     63  17.8   3.7  1.7  3.4  50.8      0.5  1.4  13.7  0.2  0.5  54.0   
...   ..   ...   ...  ...  ...   ...      ...  ...   ...  ...  ...   ...   
7995  32   9.2   1.8  0.7  1.8  40.3     -0.1 -0.2  23.1  0.4  0.6  65.7   
7996  54   6.0   1.8  0.7  1.4  48.7      0.1  0.1   3.1  0.2  0.4  70.1   
7997  85  28.2  10.7  4.0  9.0  45.1      0.2  0.6  23.6  2.8  3.9  69.7   
7998  39   7.7   2.5  1.0  2.3  40.1     -0.3 -0.5  13.3  0.6  0.7  74.3   
7999  49  19.2   4.8  1.7  5.1  32.6      0.7  2.4  41.3  0.8  1.3  70.1   

      OREB  DREB  REB  AST  STL  BLK  TOV  TARGET_5Yr

In [183]:
#For test dataframe, remove redundant column 'Id_old'
df_test.drop('Id_old', axis=1, inplace=True)
df_test.columns = df_test.columns.str.strip()
df_test.describe

<bound method NDFrame.describe of         Id  GP   MIN   PTS  FGM   FGA   FG%  3P Made  3PA   3P%  FTM  FTA  \
0        0  56   9.1   4.0  1.6   3.7  43.7      0.1  0.3   7.3  0.7  1.2   
1        1  43  19.3  10.1  3.7   8.1  46.0      0.6  1.7  35.1  1.8  2.5   
2        2  82  33.9  11.3  4.9  10.6  45.6      0.5  1.9  44.8  1.8  2.7   
3        3  86  44.7  18.8  6.8  15.9  42.9      0.5  1.8  13.5  4.5  6.3   
4        4  58  12.3   4.7  1.6   4.0  40.0      0.5  1.7  38.7  1.1  1.3   
...    ...  ..   ...   ...  ...   ...   ...      ...  ...   ...  ...  ...   
3794  3794  84  21.2   8.7  3.4   6.7  50.2      0.0  0.0  -4.5  1.7  2.5   
3795  3795  49  16.3   6.4  2.9   6.6  44.4     -0.1 -0.4  19.8  1.0  1.9   
3796  3796  53   9.9   2.1  0.8   1.8  43.1     -0.4 -0.6   1.9  0.6  1.0   
3797  3797  89  38.3  14.5  5.4  11.8  45.2      0.5  1.2  24.7  2.5  2.9   
3798  3798  55  12.0   3.0  1.1   2.5  42.3      0.1  0.6  11.7  0.9  1.1   

       FT%  OREB  DREB  REB  AST  STL  BL

## **Feature Correlation and Selection**

When there are two or more correlated features, multicollinearity can occur and this can lead to skewed or misleading results [1]. In order to reduce this from happening, the features have to be examined for colinearity. Pearson correlation was used for this analysis.

In [184]:
#Use Pearson Correlation to determine feature correlation 
pearsoncorr = df_train.corr('pearson')

#Create heatmap of pearson correlation factors
fig, ax = plt.subplots(figsize=(10,10))
sb.heatmap(pearsoncorr, 
            xticklabels=pearsoncorr.columns,
            yticklabels=pearsoncorr.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.2)

In [157]:
#Drop correlated features w/ score over 0.9 - retain "MINS", "3P MADE","FTM","REB"
cols_drop = ["PTS", "FGM", "FGA", "3PA", "FTA", "DREB", "OREB", "BLK"]
df_train.drop(cols_drop, axis=1, inplace=True)
df_test.drop(cols_drop, axis=1, inplace=True)

In [8]:
#Remove rows with negative values
#df_train = df_train[(df_train > 0).all(1)]

In [158]:
# Split the training dataset using common function data_reader.splitdata
X_train, X_val, y_train, y_val  = data_reader.split_data(df_train)

## **Model Selection and Training**

Instead of trial-and-error selection of models, a quick regression model selector was run on common models to see which would yield
the lowest mean squared error for the training/validation dataset:

From the above evaluation, it can be seen that all the models except the SVR model returned the same mean squared error (0.355). For this version, the LassoCV model was selected for evaluation. In order to boost accuracy, paramater tuning is employed on the LassoCV to find the optimal alpha value.

In [159]:
#Create Optimised XGBoost Model
optmodel = xgboost.XGBClassifier()

In [160]:
#Import hyperopt functions for hyperparameter tuning
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin
#Define objective search grid
space = {
    'max_depth' : hp.choice('max_depth', range(5, 20, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.05),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
    'subsample' : hp.quniform('subsample', 0.1, 1, 0.05),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.05)
}

In [161]:
#Define objective for optimised parameters
def objective(space):
    from sklearn.model_selection import cross_val_score
    
    optmodel = xgboost.XGBClassifier(
        max_depth = int(space['max_depth']),
        learning_rate = space['learning_rate'],
        min_child_weight = space['min_child_weight'],
        subsample = space['subsample'],
        colsample_bytree = space['colsample_bytree']
    )
    
    acc = cross_val_score(optmodel, X_train, y_train, cv=10, scoring="accuracy").mean()

    return{'loss': 1-acc, 'status': STATUS_OK }

In [162]:
#Search for best parameters
best = fmin(
    fn=objective,   
    space=space,       
    algo=tpe.suggest,       
    max_evals=7
)

[00:43:36] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
  0%|          | 0/7 [00:00<?, ?trial/s, best loss=?]

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[00:43:36] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:43:36] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:43:37] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:43:37] WARNING: /home/conda/feedstock_r

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[00:43:40] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:43:40] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:43:40] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:43:40] WARNING: /home/conda/feedstock_r

...

[00:43:46] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:43:46] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:43:47] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:43:47] WARNING: /home/conda/feedstock_r

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[00:43:50] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:43:51] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:43:51] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:43:51] WARNING: /home/conda/feedstock_r

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[00:43:52] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:43:53] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:43:53] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:43:53] WARNING: /home/conda/feedstock_r

In [163]:
#Print the optimised parameters
print("Best: ", best)

Best:  {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 8.0, 'subsample': 0.9}


In [164]:
#Create model with the optimised parameters
model = xgboost.XGBClassifier(
    max_depth = best['max_depth'],
    learning_rate = best['learning_rate'],
    min_child_weight = best['min_child_weight'],
    subsample = best['subsample'],
    colsample_bytree = best['colsample_bytree']
)

In [165]:
model.fit(X_train, y_train);

[00:44:06] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [166]:
#Store model in /models
from joblib import dump
dump(model, '../models/xgb01.joblib')

['../models/xgb01.joblib']

In [167]:
#Create predictions for train and validation
y_train_preds = model.predict(X_train)
y_val_preds = model.predict(X_val)

In [168]:
#Evaluate train predictions
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print(mse(y_train, y_train_preds, squared=False))
print(mae(y_train, y_train_preds))
accuracy_score(y_train, y_train_preds)

0.4050462936504913
0.1640625


0.8359375

In [169]:
#Train confusion matrix
confusion_matrix(y_train, y_train_preds)

array([[  39, 1035],
       [  15, 5311]])

In [145]:
#Validation set scores
y_val_prediction = model.predict(X_val)
print(mse(y_val, y_val_preds, squared=False))
print(mae(y_val, y_val_preds))
accuracy_score(y_val, y_val_preds)

0.40233692348577704
0.161875


0.838125

In [170]:
#Validation confusion matrix
confusion_matrix(y_val, y_val_prediction)

array([[   7,  250],
       [   9, 1334]])

After training and validation, the LassoCV model achieved an accuracy of 83.6% and 83.8% respectively. It is expected to do well in testing against the final target output.

## **Test output**

The output needs to be formatted to comply with the kaggle competition input requirements. The final steps detail this formatting,
as well as the outcome of the model's testing.

In [172]:
#Output predictions
df_test.drop('Id', axis=1, inplace=True)
X_test = df_test
y_test_preds = model.predict(X_test)

In [173]:
y_test_preds

array([1, 1, 1, ..., 1, 1, 1])

In [176]:
output = pd.DataFrame({'Id': range(0,3799), 'TARGET_5Yrs': [p for p in y_test_preds]})

In [178]:
output.to_csv("../reports/tay_donovan_submission_xgb02.csv", index=False)

## **Outcome**

After outputting the predictions into kaggle, the final score was 50.8%. This was not the highest achieved by Jump Shot so some further improvements can be made.